In [1]:
import pickle
# import MySQLdb
import pymysql
import numpy as np
import random
import matplotlib.pyplot as plt
import datetime
import operator as op
import math
import random
import pandas as pd
import os

In [2]:
file_path = 'D:\\jupyter files\\'
preprocess_path = file_path + 'preprocess_data_waiting_time_project'
if not os.path.exists(preprocess_path):
    os.makedirs(preprocess_path)
else:
    print(preprocess_path, 'already exists!')

D:\jupyter files\preprocess_data_waiting_time_project already exists!


In [3]:
# 保存预处理之后的数据，pickle文件
def pickle_dump(data_to_pickle, filename, local_path):
    """ Function to write data to pickle format. """
    file = os.path.join(local_path, filename)
    pickle.dump(data_to_pickle, open(file, "wb+"))

# 读取预处理之后的数据，pickle文件
def pickle_load(filename, local_path):
    """ Function to load data to pickle format. """
    file = os.path.join(local_path, filename)
    with open(file, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        data = u.load()
    return data

In [6]:
# # Connect to the MySQL database
# def getSQLtest(sql):
#     # Connect to the MySQL database
#     db = MySQLdb.connect(host='localhost',
#                          port=3306,
#                          user='root',
#                          passwd='root',
#                          db='originaldata')
#     db_cursor = db.cursor(MySQLdb.cursors.DictCursor)
#     sql0 = """
#          set sql_mode = 'STRICT_TRANS_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,NO_ENGINE_SUBSTITUTION';
#     """       
#     db_cursor.execute(sql0)
#     db_cursor.execute(sql)
#     db_results = db_cursor.fetchall()
#     db_cursor.close()
#     print ("Fetched Preliminary query results")
#     print ("Number of results: " + str(len(db_results)))
#     return db_results

In [14]:
# Connect to the MySQL database
def getSQLtest_to_df(sql):
    # Connect to the MySQL database
    db = pymysql.connect(host='localhost',
                         port=3306,
                         user='root',
                         passwd='root',
                         db='originaldata')
    db_cursor = db.cursor()
    db_cursor.execute(sql)
#     db_results = db_cursor.fetchall()
    df = pd.DataFrame(list(db_results.get('data')), columns = db_results.get('head'))
    
#     rs_df       = pd.DataFrame(list(rs.get('data')),columns=rs.get('head'))
    
    db_cursor.close()
    print ("Fetched Preliminary query results")
    print ("Number of results: ", df.shape)
    return df

In [20]:
# Connect to the MySQL database
db = pymysql.connect(host='localhost',
                     port=3306,
                     user='root',
                     passwd='root',
                     db='originaldata')
db_cursor = db.cursor()
db_cursor.execute(sql1)
db_results = db_cursor.fetchall()
# df = pd.DataFrame(list(a.get('data')), columns = a.get('head'))

# #     rs_df       = pd.DataFrame(list(rs.get('data')),columns=rs.get('head'))

# db_cursor.close()
# print ("Fetched Preliminary query results")
# print ("Number of results: ", df.shape)

AttributeError: 'int' object has no attribute 'get'

In [22]:
db_results = db_cursor.fetchall()

In [24]:
# db_results

In [15]:
# 没有年龄和性别

# Preliminary query for feature data

sql1 = """
    SELECT
        ap.AppointmentSerNum,
        ap.AliasSerNum,
        ap.PatientSerNum,
        ap.ScheduledStartTime,
        ap.ScheduledEndTime,
        ap.ActualStartDate,
        ap.ActualEndDate,
        
        res.ResourceName,
        res.ResourceType,
        
        ptlmh.PatientLocationMHSerNum,
        ptlmh.RevCount,
        ptlmh.ArrivalDateTime,
        ptlmh.HstryDateTime,
        ptlmh.checkedInFlag,
        
        ptl.ArrivalDateTime pltArrivalDateTime,
        ptl.checkedInFlag
        
    FROM
        Appointment ap
    INNER JOIN Patientlocation ptl
        ON ptl.AppointmentSerNum = ap.AppointmentSerNum
    INNER JOIN Patientlocationmh ptlmh
        ON ptlmh.AppointmentSerNum = ptl.AppointmentSerNum
    INNER JOIN Resourceappointment resapp
        ON resapp.AppointmentSerNum = ptlmh.AppointmentSerNum
    INNER JOIN Resource res
        ON res.ResourceSerNum = resapp.ResourceSerNum
    
    WHERE
        ap.ActualStartDate NOT IN ('0000-00-00 00:00:00','1970-01-01 00:00:00')
    AND ap.ActualEndDate NOT IN ('0000-00-00 00:00:00','1970-01-01 00:00:00')
    AND ap.ScheduledStartTime NOT IN ('0000-00-00 00:00:00','1970-01-01 00:00:00')
    AND ap.ScheduledEndTime NOT IN ('0000-00-00 00:00:00','1970-01-01 00:00:00')
    AND ap.ActualEndDate NOT IN ('0000-00-00 00:00:00','1970-01-01 00:00:00')
"""
# LIMIT1: 在某些情况下，如果明知道查询结果只有一个，SQL语句中使用LIMIT 1会提高查询效率

In [16]:
# print("Executing query...")
# db_timestamps = getSQLtest(sql1)
# print('Finish!')

In [17]:
# pickle_dump(db_timestamps, 'db_timestamps.p', preprocess_path)
# db_timestamps = pickle_load('db_timestamps.p', preprocess_path)

In [18]:
print('Start getting timestamps_df dataframe')
timestamps_df = getSQLtest_to_df(sql1)
print('Finish getting timestamps_df dataframe')

Start getting timestamps_df dataframe


AttributeError: 'tuple' object has no attribute 'get'

In [25]:
timestamps_df.rename(columns={
    0: 'AppointmentSerNum',
    1: 'AliasSerNum',
    2: 'PatientSerNum',
    3: 'ScheduledStartTime',
    4: 'ScheduledEndTime',
    5: 'ActualStartDate',
    6: 'ActualEndDate',
    7: 'ResourceName',
    8: 'ResourceType',
    9: 'PatientLocationMHSerNum',
    10: 'RevCount',
    11: 'ArrivalDateTime',
    12: 'HstryDateTime',
    13: 'checkedInFlag',
    14: 'pltArrivalDateTime',
    15: 'checkedInFlag'}, inplace = True)

In [28]:
# timestamps_df

In [27]:
pickle_dump(timestamps_df, 'timestamps_df.p', preprocess_path)
# timestamps_df = pickle_load('timestamps_df.p', preprocess_path)

In [ ]:
# print('The number of AppointmentSerNum for one PatientSerNum:', count_AppointmentSerNum / count_PatientSerNum)
# print('The number of PatientLocationMHSerNum for one AppointmentSerNum:', count_PatientLocationMHSerNum / count_AppointmentSerNum)
# print('The number of PatientLocationMHSerNum for one PatientSerNum:', count_PatientLocationMHSerNum / count_PatientSerNum)

In [ ]:
# print('Start getting db_timestamps dataframe')
# timestamps_dict = {}

# # Loop through each result
# count_PatientSerNum = 0
# count_AppointmentSerNum = 0
# count_PatientLocationMHSerNum = 0

# count_machine = 0
# count_auxiliary = 0
# count_doctor = 0
# count_staff = 0
# count_venue = 0
# count_none = 0

# for row in db_timestamps:
# #     count = count + 1
# #     if count % 1000 == 0:
# #         print('Getting %d db_timestamps dictionary'%count)
# #         print(row)

#     # Retrieve row params    
#     PatientSerNum = row['PatientSerNum']
#     AppointmentSerNum = row['AppointmentSerNum']
#     AliasSerNum = row['AliasSerNum']
#     ScheduledStartTime = row['ScheduledStartTime']
#     ScheduledEndTime = row['ScheduledEndTime']
#     ActualStartDate = row['ActualStartDate']
#     ActualEndDate = row['ActualEndDate']
#     ResourceName = row['ResourceName']
#     ResourceType = row['ResourceType']
#     PatientLocationMHSerNum = row['PatientLocationMHSerNum']
#     RevCount = row['RevCount']
#     ArrivalDateTime = row['ArrivalDateTime']
#     HstryDateTime = row['HstryDateTime']
#     checkedInFlag = row['checkedInFlag']
#     pltArrivalDateTime = row['pltArrivalDateTime']
#     ptlcheckedInFlag = row['ptl.checkedInFlag']
    
#     # {'', 'Auxiliary', 'Doctor', 'Machine', 'Staff', 'Venue'}
#     if ResourceType == 'Auxiliary':
#         count_auxiliary += 1
#     if ResourceType == 'Doctor':
#         count_doctor += 1
#     if ResourceType == 'Machine':
#         count_machine += 1
#     if ResourceType == 'Staff':
#         count_staff += 1
#     if ResourceType == 'Venue':
#         count_venue += 1
#     if ResourceType == '':
#         count_none += 1
        
#     if PatientSerNum not in timestamps_dict.keys():
#         count_PatientSerNum = count_PatientSerNum + 1
#         count_AppointmentSerNum = count_AppointmentSerNum + 1
#         count_PatientLocationMHSerNum = count_PatientLocationMHSerNum + 1
#         timestamps_dict[PatientSerNum] = {
#             AppointmentSerNum: {
#                 PatientLocationMHSerNum: {
#                     'ScheduledStartTime': [ScheduledStartTime],
#                     'ScheduledEndTime': [ScheduledEndTime],
#                     'ActualStartDate': [ActualStartDate],
#                     'ActualEndDate': [ActualEndDate],
#                     'ArrivalDateTime': [ArrivalDateTime],
#                     'HstryDateTime': [HstryDateTime],
#                     'pltArrivalDateTime': [pltArrivalDateTime],
#                     'AliasSerNum': [AliasSerNum],
#                     'ResourceName': [ResourceName],
#                     'ResourceType': [ResourceType],
#                     'PatientLocationMHSerNum': [PatientLocationMHSerNum],
#                     'RevCount': [RevCount],
#                     'checkedInFlag': [checkedInFlag],
#                     'ptlcheckedInFlag': [ptlcheckedInFlag]
#                 }
#             }
#         }
#         continue
        
#     if AppointmentSerNum not in timestamps_dict[PatientSerNum].keys():
#         count_AppointmentSerNum = count_AppointmentSerNum + 1
#         count_PatientLocationMHSerNum = count_PatientLocationMHSerNum + 1
#         timestamps_dict[PatientSerNum][AppointmentSerNum] = {
#             PatientLocationMHSerNum: {
#                 'ScheduledStartTime': [ScheduledStartTime],
#                 'ScheduledEndTime': [ScheduledEndTime],
#                 'ActualStartDate': [ActualStartDate],
#                 'ActualEndDate': [ActualEndDate],
#                 'ArrivalDateTime': [ArrivalDateTime],
#                 'HstryDateTime': [HstryDateTime],
#                 'pltArrivalDateTime': [pltArrivalDateTime],
#                 'AliasSerNum': [AliasSerNum],
#                 'ResourceName': [ResourceName],
#                 'ResourceType': [ResourceType],
#                 'PatientLocationMHSerNum': [PatientLocationMHSerNum],
#                 'RevCount': [RevCount],
#                 'checkedInFlag': [checkedInFlag],
#                 'ptlcheckedInFlag': [ptlcheckedInFlag]
#             }
#         }
#         continue
        
#     if PatientLocationMHSerNum not in timestamps_dict[PatientSerNum][AppointmentSerNum].keys():
#         count_PatientLocationMHSerNum = count_PatientLocationMHSerNum + 1
#         timestamps_dict[PatientSerNum][AppointmentSerNum][PatientLocationMHSerNum] = {
#             'ScheduledStartTime': [ScheduledStartTime],
#             'ScheduledEndTime': [ScheduledEndTime],
#             'ActualStartDate': [ActualStartDate],
#             'ActualEndDate': [ActualEndDate],
#             'ArrivalDateTime': [ArrivalDateTime],
#             'HstryDateTime': [HstryDateTime],
#             'pltArrivalDateTime': [pltArrivalDateTime],
#             'AliasSerNum': [AliasSerNum],
#             'ResourceName': [ResourceName],
#             'ResourceType': [ResourceType],
#             'PatientLocationMHSerNum': [PatientLocationMHSerNum],
#             'RevCount': [RevCount],
#             'checkedInFlag': [checkedInFlag],
#             'ptlcheckedInFlag': [ptlcheckedInFlag]
#         }
#         continue    

# print('Finish getting timestamps_dict dictionary')
# print('The number of count_PatientSerNum:', count_PatientSerNum)
# print('The number of count_AppointmentSerNum:', count_AppointmentSerNum)
# print('The number of count_PatientLocationMHSerNum:', count_PatientLocationMHSerNum)
# print('The number of count_machine:', count_machine)
# print('The number of count_auxiliary:', count_auxiliary)
# print('The number of count_doctor:', count_doctor)
# print('The number of count_staff:', count_staff)
# print('The number of count_venue:', count_venue)
# print('The number of count_none:', count_none)

In [ ]:
# pickle_dump(timestamps_dict, 'timestamps_dict.p', preprocess_path)
# timestamps_dict = pickle_load('timestamps_dict.p', preprocess_path)

In [ ]:
timestamps_df_grouped = timestamps_df.groupby(1)

In [13]:
timestamps_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,849926,23,38996,2015-10-19 14:00:00,2015-10-19 14:15:00,2015-10-19 14:18:00,2015-10-19 14:34:00,TB_6,Machine,290348,0,2015-10-19 13:15:00,2015-10-19 13:15:00,1,2015-10-19 14:21:00,1
1,849926,23,38996,2015-10-19 14:00:00,2015-10-19 14:15:00,2015-10-19 14:18:00,2015-10-19 14:34:00,TB_6,Machine,290349,1,2015-10-19 13:15:00,2015-10-19 14:18:00,1,2015-10-19 14:21:00,1
2,849926,23,38996,2015-10-19 14:00:00,2015-10-19 14:15:00,2015-10-19 14:18:00,2015-10-19 14:34:00,TB_6,Machine,290350,2,2015-10-19 13:15:00,2015-10-19 14:18:00,1,2015-10-19 14:21:00,1
3,849926,23,38996,2015-10-19 14:00:00,2015-10-19 14:15:00,2015-10-19 14:18:00,2015-10-19 14:34:00,TB_6,Machine,290351,3,2015-10-19 14:21:00,2015-10-19 14:21:00,1,2015-10-19 14:21:00,1
4,849926,23,38996,2015-10-19 14:00:00,2015-10-19 14:15:00,2015-10-19 14:18:00,2015-10-19 14:34:00,TB_6,Machine,290352,4,2015-10-19 14:21:00,2015-10-19 14:34:00,1,2015-10-19 14:21:00,1
5,849926,23,38996,2015-10-19 14:00:00,2015-10-19 14:15:00,2015-10-19 14:18:00,2015-10-19 14:34:00,TB_6,Machine,290353,5,2015-10-19 14:21:00,2015-10-19 14:34:00,1,2015-10-19 14:21:00,1
6,849926,23,38996,2015-10-19 14:00:00,2015-10-19 14:15:00,2015-10-19 14:18:00,2015-10-19 14:34:00,TB_6,Machine,290354,6,2015-10-19 14:21:00,2015-10-19 14:34:00,1,2015-10-19 14:21:00,1
7,849926,23,38996,2015-10-19 14:00:00,2015-10-19 14:15:00,2015-10-19 14:18:00,2015-10-19 14:34:00,TB_6,Machine,290355,7,2015-10-19 14:21:00,2015-10-19 14:34:00,1,2015-10-19 14:21:00,1
8,849926,23,38996,2015-10-19 14:00:00,2015-10-19 14:15:00,2015-10-19 14:18:00,2015-10-19 14:34:00,TB_6,Machine,290356,8,2015-10-19 14:21:00,2015-10-19 14:34:00,1,2015-10-19 14:21:00,1
9,849926,23,38996,2015-10-19 14:00:00,2015-10-19 14:15:00,2015-10-19 14:18:00,2015-10-19 14:34:00,TB_6,Machine,290357,9,2015-10-19 14:21:00,2015-10-19 14:34:00,1,2015-10-19 14:21:00,1


In [93]:
pat1 = list(timestamps_dict.keys())[0]
appt1 = list(timestamps_dict[pat1].keys())[2]

for patloc in timestamps_dict[pat1][appt1].values():
    print(patloc['RevCount'])
# pat1

# for key, pat timestamps_dict[pat1].items():
#     for appt in pat.

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]


In [92]:
timestamps_dict[pat1][appt1]

{290358: {'ScheduledStartTime': [datetime.datetime(2015, 10, 20, 13, 45)],
  'ScheduledEndTime': [datetime.datetime(2015, 10, 20, 14, 0)],
  'ActualStartDate': [datetime.datetime(2015, 10, 20, 13, 32)],
  'ActualEndDate': [datetime.datetime(2015, 10, 20, 13, 48)],
  'ArrivalDateTime': [datetime.datetime(2015, 10, 20, 12, 11)],
  'HstryDateTime': [datetime.datetime(2015, 10, 20, 12, 11)],
  'pltArrivalDateTime': [datetime.datetime(2015, 10, 20, 13, 27)],
  'AliasSerNum': [23],
  'ResourceName': ['TB_4'],
  'ResourceType': ['Machine'],
  'PatientLocationMHSerNum': [290358],
  'RevCount': [0],
  'checkedInFlag': [1],
  'ptlcheckedInFlag': [1]},
 290359: {'ScheduledStartTime': [datetime.datetime(2015, 10, 20, 13, 45)],
  'ScheduledEndTime': [datetime.datetime(2015, 10, 20, 14, 0)],
  'ActualStartDate': [datetime.datetime(2015, 10, 20, 13, 32)],
  'ActualEndDate': [datetime.datetime(2015, 10, 20, 13, 48)],
  'ArrivalDateTime': [datetime.datetime(2015, 10, 20, 13, 27)],
  'HstryDateTime': [d

In [69]:
len(timestamps_dict[38996][849926].keys())

10

In [73]:
np.mean([1,2,3])

2.0